In [ ]:
from pathlib import Path
import pandas as pd

def criar_df_estabelecimento(caminho_arquivo: Path):
    # carregando o dataframe estabelecimento
    # definindo as colunas que serão carregadas
    colunas_selecionadas = [0, 1, 2, 3, 4, 5, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 27]
    # definindo o nome das colunas
    colunas_estabelecimentos = ['CNPJ BÁSICO',
                                'CNPJ ORDEM',
                                'CNPJ DV',
                                'ID MATRIZ/FILIAL',
                                'NOME FANTASIA',
                                'SITUAÇÃO CADASTRAL',
                                'DATA INÍCIO ATIVIDADE',
                                'CNAE PRINCIPAL',
                                'CNAE SECUNDÁRIA',
                                'TIPO DE LOGRADOURO',
                                'LOGRADOURO',
                                'NÚMERO',
                                'COMPLEMENTO',
                                'BAIRRO',
                                'CEP',
                                'UF',
                                'COD_MUNICÍPIO',
                                'DDD 1',
                                'TELEFONE 1',
                                'CORREIO ELETRÔNICO']
    df_estabelecimento = pd.read_csv(
        caminho_arquivo,
        sep=';',
        encoding='latin-1',
        usecols=colunas_selecionadas,
        names=colunas_estabelecimentos,
        parse_dates=['DATA INÍCIO ATIVIDADE'],
        dtype={'CEP': 'string', 'DDD 1': 'string', 'TELEFONE 1': 'string', 'CNPJ ORDEM': 'string', 'CNPJ DV': 'string', 'CNPJ BÁSICO': 'string'},
        low_memory=True
        )
    # selecionando as empresas ativas
    df_empresas_ativos = df_estabelecimento[df_estabelecimento['SITUAÇÃO CADASTRAL'] == 2]

    # selecionando apenas as empresas do RJ
    df_empresas_ativos_rj = df_empresas_ativos[df_empresas_ativos['UF'] == 'RJ'].copy()

    # excluindo as colunas UF e SITUAÇÃO CADASTRAL
    df_empresas_ativos_rj.drop(columns=['SITUAÇÃO CADASTRAL', 'UF'], inplace=True)

    # filtrando para ter apenas empresas com telefone ou e-mail
    df_empresas_ativos_rj_filtrado = df_empresas_ativos_rj[~(df_empresas_ativos_rj['TELEFONE 1'].notnull() & df_empresas_ativos_rj['CORREIO ELETRÔNICO'].notnull())]
    
    return df_empresas_ativos_rj_filtrado
    

In [2]:
def criar_df_municipio(caminho_arquivo: Path):
    # criando um dataframe contendo os códigos e seus respectivos nomes
    dataframe_municipios = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1', names=['COD_MUNICÍPIO', 'MUNICÍPIO'], low_memory=True)
    return dataframe_municipios

In [3]:
def criar_df_empresa(caminho_arquivo: Path):
    # carregando o dataframe empresas0
    # definindo o nome das colunas
    colunas_empresas =  ['CNPJ BÁSICO', 'RAZÃO SOCIAL','CAPITAL SOCIAL', 'PORTE']
    df_empresas = pd.read_csv(
        caminho_arquivo, sep=';', encoding='latin-1', usecols=[0, 1, 4, 5], names=colunas_empresas, dtype={'CNPJ BÁSICO': 'string', 'PORTE': 'Int64'}, low_memory=True
        )
    # Definindo o padrão dos nomes dos MEIs
    padrao_regex = r'\d{2}\.\d{3}\.\d{3}|\d{11}'
    # Filtrando apenas as empresas que não são MEIs
    df_filtrado = df_empresas[~df_empresas['RAZÃO SOCIAL'].str.contains(padrao_regex, regex=True, na=False)]

    return df_filtrado

In [ ]:
from pathlib import Path

caminho = Path(r'Baixados\Selecionados')

# percorrendo pela pasta e salvando todos os dataframes em uma lista
lista_dataframe_estabelecimento = []
for pasta in caminho.iterdir():
    if 'Estabelecimentos' in pasta.name:
        # obter o nome do arquivo dentro da pasta
        for arquivo in pasta.iterdir():
            if arquivo:
                # aplicar a função
                dataframe = criar_df_estabelecimento(arquivo)
                lista_dataframe_estabelecimento.append(dataframe)

# Concatendo todos os dataframes encontrados
dataframe_final = pd.concat(lista_dataframe_estabelecimento, ignore_index=True)

In [ ]:
# dataframe_merge = pd.merge(dataframe_final, dataframe_municipios, on='COD_MUNICÍPIO', how='left')
# dataframe_merge.drop(columns=['COD_MUNICÍPIO'], inplace=True)
# ordem_colunas = [               'CNPJ BÁSICO',
#                                 'CNPJ ORDEM',
#                                 'CNPJ DV',
#                                 'ID MATRIZ/FILIAL',
#                                 'NOME FANTASIA',
#                                 'DATA INÍCIO ATIVIDADE',
#                                 'CNAE PRINCIPAL',
#                                 'CNAE SECUNDÁRIA',
#                                 'TIPO DE LOGRADOURO',
#                                 'LOGRADOURO',
#                                 'NÚMERO',
#                                 'COMPLEMENTO',
#                                 'BAIRRO',
#                                 'CEP',
#                                 'MUNICÍPIO',
#                                 'DDD 1',
#                                 'TELEFONE 1',
#                                 'CORREIO ELETRÔNICO']
# dataframe_merge = dataframe_merge.reindex(ordem_colunas, axis=1)

In [4]:
# obter lista de dataframe empresas
# obter lista de dataframe estabelecimentos
def obter_lista_dataframe(caminho: Path, nome_tipo: str):
    # percorrendo pela pasta e salvando todos os dataframes em uma lista
    lista_dataframe = []
    for pasta in caminho.iterdir():
        if nome_tipo in pasta.name:
            # obter o nome do arquivo dentro da pasta
            for arquivo in pasta.iterdir():
                if arquivo:
                    # aplicar a função
                    if nome_tipo == 'Estabelecimento':
                        dataframe = criar_df_estabelecimento(arquivo)
                    else:
                        dataframe = criar_df_empresa(arquivo)
                    lista_dataframe.append(dataframe)

    # Concatendo todos os dataframes encontrados
    dataframe_concatenado = pd.concat(lista_dataframe, ignore_index=True)
    return dataframe_concatenado

In [5]:
dataframe_estabelecimento = obter_lista_dataframe(Path(r'Baixados\Selecionados'), 'Estabelecimento')

In [7]:
dataframe_empresa = obter_lista_dataframe(Path(r'Baixados\Selecionados'), 'Empresa')

In [20]:
# mesclando os arquivos
display(dataframe_empresa.head(10))

,CNPJ BÁSICO,RAZÃO SOCIAL,CAPITAL SOCIAL,PORTE
0,41273588,STUDIO MVO2 LTDA,"20000,00",1
1,41273590,MARIA DAS MERCES SOARES LEMOS,"0,00",5
2,41273603,GRAFLINE ACESSORIOS GRAFICOS LTDA,"10000,00",1
3,41273604,RUMO - ESTUDIO DE DANCA LTDA,"10000,00",1
4,41273608,FRANCISCA SAMPAIO DE OLIVEIRA,"0,00",1
5,41273612,ELLEN FABIANI SOUZA DA SILVA AGENCIA DE PUBLIC...,"1000,00",1
6,41273614,SERGIO BUENO DE FREITAS,"4000,00",1
7,41273616,ANTONIO MORAIS DA SILVA,"0,00",1
8,41273624,FRANCISCA MIRANDA OLIVEIRA,"0,00",1
9,41273625,MILTON FEITOSA CARDOSO CERAMICA LTDA,"110000,00",1


In [22]:
display(dataframe_estabelecimento.head(10))

,CNPJ BÁSICO,CNPJ ORDEM,CNPJ DV,ID MATRIZ/FILIAL,NOME FANTASIA,DATA INÍCIO ATIVIDADE,CNAE PRINCIPAL,CNAE SECUNDÁRIA,TIPO DE LOGRADOURO,LOGRADOURO,NÚMERO,COMPLEMENTO,BAIRRO,CEP,COD_MUNICÍPIO,DDD 1,TELEFONE 1,CORREIO ELETRÔNICO
0,29794219,0001,91,1,NaN,2018-02-27 00:00:00,8219999,NaN,RUA,CLAUDE BERNARD,125,CASA 01,CAMPO GRANDE,23045820,6001,21,35963024,martasmachado@gmail.com
1,29568497,0001,20,1,NaN,2018-01-30 00:00:00,9602501,NaN,RUA,FERNANDO DA COSTA,132,LOJA ;QUADRA 3;LOTE 27,LOTE XV - Pq MARTINHO,26183543,2909,21,91947119,blancammmfley@hotmail.com
2,29794924,0001,99,1,NaN,2018-02-27 00:00:00,3299099,NaN,RUA,TRES,33,NaN,CASA DE PEDRA,27259060,5925,24,88331646,TBPSTHAIS@YAHOO.COM.BR
3,29795175,0001,14,1,PATINHAS AO SOL,2017-07-07 00:00:00,9499500,NaN,RUA,PROJETADA,20,NaN,JARDIM ESPERANCA,28924030,5813,21,64669980,CARS21031804@GMAIL.COM
4,29795198,0001,29,1,NaN,2018-02-27 00:00:00,9602502,NaN,RUA,JACUPEMA,S/N,SALA 2,OLARIA,21071140,6001,21,36865815,rosangelacristina1993@gmail.com
5,29795516,0001,51,1,NaN,2018-02-27 00:00:00,7319002,NaN,AVENIDA,DOM HELDER CAMARA,2575,LOJA B,JACAREZINHO,21050453,6001,21,31284427,FABIOGIMEN@GMAIL.COM
6,29795667,0001,00,1,NaN,2018-02-27 00:00:00,7319002,"4929901,4929902",ESTRADA,DO CAFUNDA,1757,APT 516;BLOCO 09,TAQUARA,22725030,6001,21,41043474,andre.cruz1969@gmail.com
7,29796278,0001,07,1,NaN,2018-02-27 00:00:00,4399103,"4330404,4744099",RUA,DEZ DE OUTUBRO,SN,LOTE JARDIM DO EDEN,AMPARO,28605550,5867,22,99958987,JUNIORCASAEINSTALACAO@GMAIL.COM
8,29796318,0001,02,1,NaN,2018-02-27 00:00:00,9602501,NaN,RUA,WALDIR HERDY GEVAUX,99,NaN,RECANTO DAS PALMEIRAS 2,28540000,5829,22,88578514,TAVARESPRISCILA049@GMAIL.COM
9,29796472,0001,84,1,NaN,2018-02-27 00:00:00,9602501,8230001,RUA,3,819,NaN,NOVA CAMPINAS,25268060,5833,21,68658828,ISABELAA22009@GMAIL.COM


In [30]:
df_estabelecimento_filtrado = dataframe_estabelecimento[~(dataframe_estabelecimento['TELEFONE 1'].notnull() & dataframe_estabelecimento['CORREIO ELETRÔNICO'].notnull())]


In [33]:
df_merge = pd.merge(df_estabelecimento_filtrado, dataframe_empresa, on='CNPJ BÁSICO', how='inner')

In [34]:
display(df_merge.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256394 entries, 0 to 256393
Data columns (total 21 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   CNPJ BÁSICO            256394 non-null  string
 1   CNPJ ORDEM             256394 non-null  string
 2   CNPJ DV                256394 non-null  string
 3   ID MATRIZ/FILIAL       256394 non-null  int64 
 4   NOME FANTASIA          100275 non-null  object
 5   DATA INÍCIO ATIVIDADE  256394 non-null  object
 6   CNAE PRINCIPAL         256394 non-null  int64 
 7   CNAE SECUNDÁRIA        120251 non-null  object
 8   TIPO DE LOGRADOURO     253005 non-null  object
 9   LOGRADOURO             256394 non-null  object
 10  NÚMERO                 256394 non-null  object
 11  COMPLEMENTO            150150 non-null  object
 12  BAIRRO                 256161 non-null  object
 13  CEP                    256393 non-null  string
 14  COD_MUNICÍPIO          256394 non-null  int64 
 15  

None

In [35]:
df_merge.to_excel('Teste.xlsx')

In [21]:
display(df_merge.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833783 entries, 0 to 833782
Data columns (total 21 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   CNPJ BÁSICO            833783 non-null  string
 1   CNPJ ORDEM             833783 non-null  string
 2   CNPJ DV                833783 non-null  string
 3   ID MATRIZ/FILIAL       833783 non-null  int64 
 4   NOME FANTASIA          438751 non-null  object
 5   DATA INÍCIO ATIVIDADE  833783 non-null  object
 6   CNAE PRINCIPAL         833783 non-null  int64 
 7   CNAE SECUNDÁRIA        518056 non-null  object
 8   TIPO DE LOGRADOURO     829978 non-null  object
 9   LOGRADOURO             833781 non-null  object
 10  NÚMERO                 833779 non-null  object
 11  COMPLEMENTO            578685 non-null  object
 12  BAIRRO                 833542 non-null  object
 13  CEP                    833782 non-null  string
 14  COD_MUNICÍPIO          833783 non-null  int64 
 15  

None